# Classificação dos tipos de cogumelo

O objetivo desse projeto é conseguir classificar um cogumelo como comestível ou venenoso.

- Esther Dagir
- Gustavo Katz
- Lila Habda
- Rafael Oncins

### Importando bibliotecas e fazendo a leitura do Dataframe

In [ ]:
import pandas as pd
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, probplot
from sklearn import tree

#leitura do Dataframe
df = pd.read_csv('mushrooms.csv')

In [ ]:
# Dataframe dos cogumelos
df.head(5)

## Análise exploratória

A ideia nessa análise é avaliar a influência de cada categoria na determinação de um cogumelo como comestível ou venenoso. Para isso, vamos criar gráficos que analisam a frequência de determinada característica em cada tido de cogumelo, conforme cada categoria (coluna).

In [ ]:
# Para construir um gráfico para cada coluna, vamos dividir em três células a fim de facilitarr a visualização
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

### Análise inicial
A partir da análise dos gráficos, foi possível identificar as categorias que mais influenciam a determinação de um cogumelo como comestível ou venenoso. Dessa maneira, filtramos algumas categorias:

odor, stalk-color-above-ring, ring-type, spore-print-color, population, habitat

A partir desse ponto, nós vamos utilizar apenas essas colunas como referências para o nosso modelo. Portanto, criamos um novo dataframe:

In [ ]:
# Separando somente as colunas que serão utilizadas na classificação
cl_utilizadas = df.loc[:,['odor','stalk-color-above-ring','ring-type','spore-print-color','population','habitat']]
cl_utilizadas

In [ ]:
# Trocando as variáveis letras para números, para conseguir utilizar essa biblioteca
x = pd.get_dummies(cl_utilizadas)

df['class'] = df['class'].replace('e', 0)
df['class'] = df['class'].replace('p', 1)
y = df2['class']

In [ ]:
# árvore de decisão
# melhor maneira de classificar um cogumelo como comestível ou venenoso, com base nas características selecionadas acima
clf = tree.DecisionTreeClassifier(random_state = 0)
clf = clf.fit(x, df['class'])
tree.plot_tree(clf)

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, feature_names=x.columns, filled=True)

Além dos gráficos, montamos uma árvore de classificação para ajudar na visualização de como essas categorias que selecionamos influenciam na separação dos cogumelos. 

In [ ]:
# Separando os dados em treinamento e teste
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2)

## Primeiro modelo

Nesse primeiro modelo de predição, utilizamos as categorias que mais influenciam para criar um classificador. Assim, a biblioteca sklearn utilizada separa os dados em uma parte de treinamento e outra de teste. Ao se ajustar com o treinamento, o classificador consegue avaliar a parte de teste e trazer resultados precisos.

In [ ]:
# Aplicando o classificador
from sklearn.ensemble import ExtraTreesClassifier

modelo1 = ExtraTreesClassifier(n_estimators = 100) 
M1 = modelo1.fit(x_treino, y_treino) # modelo ajustado conforme a planilha de treinamento

resultado_treino = M1.score(x_treino, y_treino) # resutado da aplicação do classificador na própria planilha de treinamento
resultado_teste = M1.score(x_teste, y_teste) # resultado obtido com a planilha teste

print(f'Acurácia do treino:{resultado_treino}')
print(f'Acurácia do teste: {resultado_teste}')

## Segundo modelo

No segundo modelo, utilizamos a mesma biblioteca sklearn, e os mesmos dados de treinamento e teste, a fim de ser uma melhor maneira de comparar os resultados no final. Nesse caso, utilizamos a técnica da regressão logística para gerar um classificador.

In [ ]:
from sklearn.linear_model import LogisticRegression

modelo2= LogisticRegression(fit_intercept = False)
M2 = modelo2.fit(x_treino, y_treino)

resultado_treino2 = M2.score(x_treino, y_treino) # resutado da aplicação do classificador na própria planilha de treinamento
resultado_teste2 = M2.score(x_teste, y_teste) # resultado obtido com a planilha teste

print(f'Acurácia do treino:{resultado_treino2}')
print(f'Acurácia do teste: {resultado_teste2}')

- Explicação melhor sobre árvore de decisão( o que é, link de referência)
- Explicação do porquê escolher aquelas categorias
- Verificar e validar modelos
- Relatório (explicação do passo a passo, descrever tudo, conclusão)

## Outras tentativas

In [ ]:
##trasformar valores escritos em valores numericos
##edible=e=0, poisonous=p=1
#df['class'] = df['class'].replace('e', 0)
#df['class'] = df['class'].replace('p', 1)
#
## bell=b=0,conical=c=1,convex=x=2,flat=f=3, knobbed=k=4,sunken=s=5
#df['cap-shape'] = df['cap-shape'].replace('b', 0)
#df['cap-shape'] = df['cap-shape'].replace('c', 1)
#df['cap-shape'] = df['cap-shape'].replace('x', 2)
#df['cap-shape'] = df['cap-shape'].replace('f', 3)
#df['cap-shape'] = df['cap-shape'].replace('k', 4)
#df['cap-shape'] = df['cap-shape'].replace('s', 5)
#
## fibrous=f=0,grooves=g=1,scaly=y=2,smooth=s=3
#df['cap-surface'] = df['cap-surface'].replace('f',0)
#df['cap-surface'] = df['cap-surface'].replace('g',1)
#df['cap-surface'] = df['cap-surface'].replace('y',2)
#df['cap-surface'] = df['cap-surface'].replace('s',3)
#
## brown=n=0,buff=b=1,cinnamon=c=2,gray=g=3,green=r=4,pink=p=5,purple=u=6,red=e=7,white=w=8,yellow=y=9
#df['cap-color'] = df['cap-color'].replace('n',0)
#df['cap-color'] = df['cap-color'].replace('b',1)
#df['cap-color'] = df['cap-color'].replace('c',2)
#df['cap-color'] = df['cap-color'].replace('g',3)
#df['cap-color'] = df['cap-color'].replace('r',4)
#df['cap-color'] = df['cap-color'].replace('p',5)
#df['cap-color'] = df['cap-color'].replace('u',6)
#df['cap-color'] = df['cap-color'].replace('e',7)
#df['cap-color'] = df['cap-color'].replace('w',8)
#df['cap-color'] = df['cap-color'].replace('y',9)
#
##bruises=t=0, no=f=1
#df['bruises'] = df['bruises'].replace('t',0)
#df['bruises'] = df['bruises'].replace('f',1)
#
## almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
#df['odor'] = df['odor'].replace('a',0)
#df['odor'] = df['odor'].replace('l',1)
#df['odor'] = df['odor'].replace('c',2)
#df['odor'] = df['odor'].replace('y',3)
#df['odor'] = df['odor'].replace('f',4)
#df['odor'] = df['odor'].replace('m',5)
#df['odor'] = df['odor'].replace('n',6)
#df['odor'] = df['odor'].replace('p',7)
#df['odor'] = df['odor'].replace('s',8)
#
## attached=a, descending=d, free=f, notched=n
#df['gill-attachment'] = df['gill-attachment'].replace('a',0)
#df['gill-attachment'] = df['gill-attachment'].replace('d',1)
#df['gill-attachment'] = df['gill-attachment'].replace('f',2)
#df['gill-attachment'] = df['gill-attachment'].replace('n',3)
#
##close=c,crowded=w,distant=d
#df['gill-spacing'] = df['gill-spacing'].replace('c', 0)
#df['gill-spacing'] = df['gill-spacing'].replace('w', 1)
#df['gill-spacing'] = df['gill-spacing'].replace('d', 2)
#
##broad=b,narrow=n
#df['gill-size'] = df['gill-size'].replace('b',0)
#df['gill-size'] = df['gill-size'].replace('n',1)
#
##black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
#df['gill-color'] = df['gill-color'].replace('k',0)
#df['gill-color'] = df['gill-color'].replace('n',1)
#df['gill-color'] = df['gill-color'].replace('b',2)
#df['gill-color'] = df['gill-color'].replace('h',3)
#df['gill-color'] = df['gill-color'].replace('g',4)
#df['gill-color'] = df['gill-color'].replace('r',5)
#df['gill-color'] = df['gill-color'].replace('o',6)
#df['gill-color'] = df['gill-color'].replace('p',7)
#df['gill-color'] = df['gill-color'].replace('u',8)
#df['gill-color'] = df['gill-color'].replace('e',9)
#df['gill-color'] = df['gill-color'].replace('w',10)
#df['gill-color'] = df['gill-color'].replace('y',11)
#
#df